In [2]:
import pandas as pd
import glob

# 병합할 파일 목록 (파일 이름 패턴을 사용하거나, 리스트로 직접 지정)
# 4개의 파일 이름이 query (숫자).csv 형태이므로 glob을 사용하면 편리합니다.
file_list = ['query (7).csv', 'query (6).csv', 'query (5).csv', 'query (4).csv']

# 또는 glob.glob("query (*).csv") 를 사용할 수도 있습니다.

# 유지할 컬럼 목록
columns_to_keep = ['time', 'latitude', 'longitude', 'depth', 'nst', 'gap', 'mag']

# 각 파일을 읽어와서 리스트에 저장
all_dataframes = []
for filename in file_list:
    try:
        df = pd.read_csv(filename)

        # 파일에 필요한 컬럼이 모두 있는지 확인
        if all(col in df.columns for col in columns_to_keep):
            # 필요한 컬럼만 선택
            df_selected = df[columns_to_keep]
            all_dataframes.append(df_selected)
            print(f"'{filename}' 처리 완료 (행: {len(df_selected)})")
        else:
            print(f"경고: '{filename}'에 필요한 컬럼이 부족하여 건너뜁니다.")

    except Exception as e:
        print(f"'{filename}' 처리 중 오류 발생: {e}")

# 모든 데이터프레임을 하나로 합치기 (위아래로 붙이기)
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)

    # 결과 파일로 저장
    output_filename = 'combined_earthquake_data.csv'
    combined_df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n총 {len(combined_df)}개의 행이 '{output_filename}' 파일로 저장되었습니다.")
else:
    print("\n병합할 데이터가 없습니다. 파일 이름이나 컬럼명을 확인해주세요.")

'query (7).csv' 처리 완료 (행: 6768)
'query (6).csv' 처리 완료 (행: 6323)
'query (5).csv' 처리 완료 (행: 5393)
'query (4).csv' 처리 완료 (행: 4023)

총 22507개의 행이 'combined_earthquake_data.csv' 파일로 저장되었습니다.


In [3]:
import pandas as pd

# 입력 파일 (TSV)
input_file = 'tsunamis-2025-11-03_13-35-46_+0900.tsv'

# 출력 파일 (CSV)
output_file = 'renamed_tsunami_data.csv'

# 변경할 컬럼 이름 맵핑
# { '기존 컬럼명' : '새 컬럼명' }
rename_map = {
    'Earthquake Magnitude': 'mag',
    'Tsunami Cause Code': 'tsunami',
    'Latitude': 'latitude',
    'Longitude': 'longitude'
}

try:
    # 1. TSV 파일을 읽어옵니다. (구분자=탭)
    #    (파일 구조 확인 후 header=0, skiprows=[1] 사용)
    df = pd.read_csv(input_file, sep='\t', header=0, skiprows=[1])

    # 컬럼 이름 앞뒤에 붙어있을 수 있는 따옴표 제거
    df.columns = df.columns.str.strip('"')

    # 2. 지정된 컬럼명 변경
    df.rename(columns=rename_map, inplace=True)

    # 3. 변경된 DataFrame을 새 CSV 파일로 저장 (기본값=쉼표 구분)
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"파일 처리 완료! '{output_file}' (CSV 파일)로 저장되었습니다.")

except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다. 파일이 코드와 같은 위치에 있는지 확인하세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

파일 처리 완료! 'renamed_tsunami_data.csv' (CSV 파일)로 저장되었습니다.


In [4]:
import pandas as pd

# 입력 파일
input_file = 'renamed_tsunami_data.csv'
# 저장할 파일
output_file = 'filtered_tsunami_data.csv'

# 1. 유지할 컬럼 목록
columns_to_keep = [
    'Year',
    'Mo',
    'tsunami',
    'mag',
    'latitude',
    'longitude',
    'Tsunami Event Validity'
]

# 2. 필터링할 컬럼과 값
filter_column = 'Tsunami Event Validity'
valid_values = [3, 4] # 3 또는 4 인 값만 유지

try:
    # 파일 읽기
    df = pd.read_csv(input_file)

    # 3. 지정된 컬럼만 선택 (유지)
    # 리스트에 없는 모든 컬럼은 삭제됩니다.
    df_selected = df[columns_to_keep]

    # 4. 'Tsunami Event Validity' 컬럼 값이 3 또는 4인 행만 필터링
    # .isin() 함수를 사용하여 [3, 4] 리스트에 포함된 값만 확인
    df_filtered = df_selected[df_selected[filter_column].isin(valid_values)]

    # 5. 새 CSV 파일로 저장
    df_filtered.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"처리 완료! '{output_file}'로 저장되었습니다.")
    print(f"원본 행: {len(df)}, 컬럼 선택 후: {len(df_selected)}, 최종 필터링 후: {len(df_filtered)}")
    print("\n필터링된 데이터 (상위 5개):")
    print(df_filtered.head())

except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다.")
except KeyError as e:
    # columns_to_keep 리스트에 없는 컬럼명을 요청했을 때 발생
    print(f"오류: {e} 컬럼을 찾을 수 없습니다. 컬럼명을 확인하세요.")
except Exception as e:
    print(f"알 수 없는 오류가 발생했습니다: {e}")

처리 완료! 'filtered_tsunami_data.csv'로 저장되었습니다.
원본 행: 135, 컬럼 선택 후: 135, 최종 필터링 후: 130

필터링된 데이터 (상위 5개):
   Year  Mo  tsunami  mag  latitude  longitude  Tsunami Event Validity
0  2001   1        1  7.7    13.049    -88.660                       3
1  2001   6        1  8.4   -16.265    -73.641                       4
2  2001  10        1  6.1    52.630   -132.200                       4
3  2001  12        1  6.8    23.954    122.734                       4
4  2002   1        1  7.2   -17.600    167.856                       4


In [5]:
import pandas as pd

# 입력 파일
input_file = 'filtered_tsunami_data.csv'
# 새 출력 파일
output_file = 'final_tsunami_data.csv'

try:
    # 1. CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 2. 'Mo' 컬럼을 'Month'로 이름 변경
    df.rename(columns={'Mo': 'Month'}, inplace=True)

    # 3. 새 CSV 파일로 저장
    df.to_csv(output_file, index=False, encoding='utf-8-sig')

    print(f"컬럼명 변경 완료! '{output_file}' 로 저장되었습니다.")
    print("\n변경된 데이터 확인 (상위 5개):")
    print(df.head())

except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

컬럼명 변경 완료! 'final_tsunami_data.csv' 로 저장되었습니다.

변경된 데이터 확인 (상위 5개):
   Year  Month  tsunami  mag  latitude  longitude  Tsunami Event Validity
0  2001      1        1  7.7    13.049    -88.660                       3
1  2001      6        1  8.4   -16.265    -73.641                       4
2  2001     10        1  6.1    52.630   -132.200                       4
3  2001     12        1  6.8    23.954    122.734                       4
4  2002      1        1  7.2   -17.600    167.856                       4


In [7]:
import pandas as pd

# 입력 파일
input_file = 'final_tsunami_data.csv'
# 새 출력 파일
output_file = 'real_final_tsunami_data.csv'
# 삭제할 컬럼 이름
column_to_drop = 'Tsunami Event Validity'

try:
    # 1. CSV 파일 읽기
    df = pd.read_csv(input_file)

    # 2. 'time' 컬럼이 있는지 확인
    if column_to_drop in df.columns:
        # 3. 'time' 컬럼 삭제
        # df.drop()을 사용하고 axis=1 (컬럼 기준)을 지정합니다.
        df_dropped = df.drop(columns=[column_to_drop])

        # 4. 새 CSV 파일로 저장
        df_dropped.to_csv(output_file, index=False, encoding='utf-8-sig')

        print(f"'{column_to_drop}' 컬럼을 성공적으로 삭제했습니다.")
        print(f"새 파일: '{output_file}'")
        print("\n삭제 후 컬럼 목록:")
        print(df_dropped.columns.to_list())

    else:
        print(f"오류: '{column_to_drop}' 컬럼이 파일에 없습니다.")

except FileNotFoundError:
    print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

'Tsunami Event Validity' 컬럼을 성공적으로 삭제했습니다.
새 파일: 'real_final_tsunami_data.csv'

삭제 후 컬럼 목록:
['Year', 'Month', 'tsunami', 'mag', 'latitude', 'longitude']


In [10]:
import pandas as pd

# --- 파일 설정 ---
# 비교할 두 파일
file1_earthquake = 'combined_earthquake_data.csv'
file2_tsunami = 'real_final_tsunami_data.csv'

# 결과(공통 행)를 저장할 파일
output_file = 'common_events_matched.csv'

# 비교할 기준이 되는 컬럼들
merge_columns = ['mag', 'latitude', 'longitude']

# 부동소수점(소수점) 비교를 위한 반올림 자릿수
# (예: 5.12와 5.120을 같게 처리하기 위함)
precision = 3
# --- --- ---

try:
    # 1. 두 CSV 파일 읽기
    df_eq = pd.read_csv(file1_earthquake)
    df_tsu = pd.read_csv(file2_tsunami)

    print(f"'{file1_earthquake}' 파일 로드 (총 {len(df_eq)} 행)")
    print(f"'{file2_tsunami}' 파일 로드 (총 {len(df_tsu)} 행)")

    # 2. 두 파일에 필요한 컬럼이 모두 있는지 확인
    missing_eq = [col for col in merge_columns if col not in df_eq.columns]
    missing_tsu = [col for col in merge_columns if col not in df_tsu.columns]

    error_found = False
    if missing_eq:
        print(f"오류: '{file1_earthquake}' 파일에 다음 컬럼이 없습니다: {missing_eq}")
        error_found = True
    if missing_tsu:
        print(f"오류: '{file2_tsunami}' 파일에 다음 컬럼이 없습니다: {missing_tsu}")
        error_found = True

    if not error_found:
        # 3. 비교 전 데이터 처리 (부동소수점 오차 방지)
        # 'mag', 'latitude', 'longitude' 컬럼을 숫자형으로 변환하고
        # 지정된 자릿수(3)에서 반올림하여 정확한 비교가 가능하게 함
        print(f"\n비교를 위해 기준 컬럼 {merge_columns} 값을 소수점 {precision}째 자리까지 표준화합니다...")

        # 원본을 유지하기 위해 .copy() 사용
        df_eq_processed = df_eq.copy()
        df_tsu_processed = df_tsu.copy()

        for col in merge_columns:
            # errors='coerce'는 숫자로 변환할 수 없는 값을 NaN(결측치)로 만듭니다.
            df_eq_processed[col] = pd.to_numeric(df_eq_processed[col], errors='coerce').round(precision)
            df_tsu_processed[col] = pd.to_numeric(df_tsu_processed[col], errors='coerce').round(precision)

        # 비교 컬럼 중 하나라도 NaN인 행은 삭제
        df_eq_processed.dropna(subset=merge_columns, inplace=True)
        df_tsu_processed.dropna(subset=merge_columns, inplace=True)

        # 4. 'inner' 조인(merge) 수행
        # 'how='inner'' 옵션은 두 데이터프레임에서
        # 'on=merge_columns' (mag, latitude, longitude) 값이 모두 일치하는 행만 남깁니다.
        # (이것이 SQL의 INNER JOIN과 동일한 기능입니다)
        common_events_df = pd.merge(
            df_eq_processed,
            df_tsu_processed,
            on=merge_columns,
            how='inner',
            suffixes=('_eq', '_tsu') # 두 파일에 이름이 겹치는 다른 컬럼(예: depth)을 구분하기 위함
        )

        # 5. 결과 저장 및 보고
        if len(common_events_df) > 0:
            common_events_df.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f"\n처리 완료! {len(common_events_df)} 개의 공통 행을 찾았습니다.")
            print(f"'{output_file}' 파일로 저장되었습니다.")
        else:
            print(f"\n두 파일 간에 {merge_columns} 값이 모두 일치하는 공통 행을 찾지 못했습니다.")

except FileNotFoundError as e:
    print(f"오류: 파일을 찾을 수 없습니다. {e}")
except Exception as e:
    print(f"알 수 없는 오류가 발생했습니다: {e}")

'combined_earthquake_data.csv' 파일 로드 (총 22507 행)
'real_final_tsunami_data.csv' 파일 로드 (총 130 행)

비교를 위해 기준 컬럼 ['mag', 'latitude', 'longitude'] 값을 소수점 3째 자리까지 표준화합니다...

처리 완료! 91 개의 공통 행을 찾았습니다.
'common_events_matched.csv' 파일로 저장되었습니다.
